<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Stock_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# import libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [3]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-3*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [4]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv')

mypf = mypf[mypf['InPortfolio'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()
stock_ids

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


array(['AMBUJACEM', 'ASIANPAINT', 'BAJFINANCE', 'BANDHANBNK', 'BATAINDIA',
       'DMART', 'EICHERMOT', 'GICRE', 'GILLETTE', 'GLAND', 'HDFCAMC',
       'ICICIGI', 'INDIGOPNTS', 'INFY', 'IRCTC', 'JMFINANCIL', 'JUBLFOOD',
       'LUXIND', 'NIACL', 'PAGEIND', 'PIDILITIND', 'PNB', 'RELAXO',
       'RELIANCE', 'TATACONSUM', 'TEAMLEASE', 'UJJIVANSFB', 'VAIBHAVGBL',
       'VALIANTORG', 'WIPRO'], dtype=object)

In [5]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'
    
    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='3y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    
    stock_df['Close'] = round(stock_df['Close'],0)     
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'
                   ], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    #current_pe = get_current_pe(stock_id)
    stock_id = stock_id.upper() + '.NS'    
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]   
    tmp['Criteria'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0]   
    tmp['Strategy'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]   
    #tmp['Curr_PE'] = current_pe
    df_prec_dev = df_prec_dev.append(tmp)

In [6]:
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

In [8]:
# portfolio stocks

tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df['Investment'] = tmp_df['Avg_Cost'] * tmp_df['Shares']
tmp_df['Current'] = tmp_df['Close'] * tmp_df['Shares']
tmp_df['Estimated'] = tmp_df['Max'] * tmp_df['Shares']
tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df['Estimated P/L%'] = round((tmp_df['Estimated'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df['Juice Left%'] = tmp_df['Estimated P/L%'] - tmp_df['Current P/L%'] 

investment = round(sum(tmp_df['Avg_Cost']*tmp_df['Shares']),0)
current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
curr_pnl_amount = current-investment
curr_pnl_percentage = round((current-investment)*100/investment,2)

estimate = round(sum(tmp_df['Max']*tmp_df['Shares']),0)
est_pnl_amount = estimate-investment
est_pnl_percentage = round((estimate-investment)*100/investment,2)

tmp_df['Weightage%'] = round(tmp_df['Investment']*100/investment,2)

print('Investment: ', investment)
print('Current: ', current)
print('Current PnL: ', curr_pnl_amount)
print('Current PnL %: ', curr_pnl_percentage)
print('Estimate: ', estimate)
print('Estimate PnL: ', est_pnl_amount)
print('Estimate PnL %: ', est_pnl_percentage)

qualified stocks: 30
Investment:  1641874.0
Current:  1652985.0
Current PnL:  11111.0
Current PnL %:  0.68
Estimate:  2879832.0
Estimate PnL:  1237958.0
Estimate PnL %:  75.4


In [9]:
tmp_df.sort_values(by = 'Current P/L%', ascending=False)

,Close,20_SMA,50_SMA,200_SMA,Stock,Max,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Estimated,Current P/L%,Estimated P/L%,Juice Left%,Weightage%
12,1409.0,1255.0,1133.0,1312.0,INDIGOPNTS.NS,3112.0,1312.07,166.0,V40N,2T3Y,217803.62,233894.0,516592.0,7.39,137.18,129.79,13.27
2,6793.0,6477.0,6073.0,6611.0,BAJFINANCE.NS,7901.0,6365.23,16.0,V40,ATH,101843.68,108688.0,126416.0,6.72,24.13,17.41,6.20
6,3545.0,3422.0,3199.0,3328.0,EICHERMOT.NS,3850.0,3336.21,15.0,V40N,ATH,50043.15,53175.0,57750.0,6.26,15.40,9.14,3.05
19,41788.0,41341.0,39090.0,44208.0,PAGEIND.NS,53631.0,39840.50,2.0,V40,ATH,79681.00,83576.0,107262.0,4.89,34.61,29.72,4.85
1,3085.0,3017.0,2897.0,3072.0,ASIANPAINT.NS,3566.0,2954.17,17.0,V40,ATH,50220.89,52445.0,60622.0,4.43,20.71,16.28,3.06
20,2535.0,2488.0,2406.0,2545.0,PIDILITIND.NS,2902.0,2442.09,20.0,V40,ATH,48841.80,50700.0,58040.0,3.80,18.83,15.03,2.97
0,424.0,399.0,386.0,456.0,AMBUJACEM.NS,588.0,408.85,120.0,TBD,ATH,49062.00,50880.0,70560.0,3.71,43.82,40.11,2.99
29,396.0,383.0,376.0,395.0,WIPRO.NS,713.0,382.90,130.0,2T3Y,2T3Y,49777.00,51480.0,92690.0,3.42,86.21,82.79,3.03
13,1293.0,1257.0,1336.0,1474.0,INFY.NS,1898.0,1252.12,40.0,V40,ATH,50084.80,51720.0,75920.0,3.26,51.58,48.32,3.05
4,1531.0,1506.0,1452.0,1652.0,BATAINDIA.NS,2173.0,1501.51,67.0,V40,ATH,100601.17,102577.0,145591.0,1.96,44.72,42.76,6.13


In [10]:
tmp_df.sort_values(by = 'Estimated P/L%', ascending=False)

,Close,20_SMA,50_SMA,200_SMA,Stock,Max,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Estimated,Current P/L%,Estimated P/L%,Juice Left%,Weightage%
28,474.0,495.0,461.0,574.0,VALIANTORG.NS,1851.0,481.73,64.0,2T3Y,2T3Y,30830.72,30336.0,118464.0,-1.60,284.24,285.84,1.88
9,894.0,1328.0,1283.0,1700.0,GLAND.NS,4316.0,1354.78,37.0,2T3Y,2T3Y,50126.86,33078.0,159692.0,-34.01,218.58,252.59,3.05
17,1445.0,1435.0,1324.0,1581.0,LUXIND.NS,4546.0,1499.87,34.0,V40N,2T3Y,50995.58,49130.0,154564.0,-3.66,203.09,206.75,3.11
27,300.0,313.0,306.0,321.0,VAIBHAVGBL.NS,966.0,320.64,96.0,2T3Y,2T3Y,30781.44,28800.0,92736.0,-6.44,201.27,207.71,1.87
12,1409.0,1255.0,1133.0,1312.0,INDIGOPNTS.NS,3112.0,1312.07,166.0,V40N,2T3Y,217803.62,233894.0,516592.0,7.39,137.18,129.79,13.27
25,2235.0,2163.0,2242.0,2654.0,TEAMLEASE.NS,5281.0,2239.86,14.0,V40N,NaN,31358.04,31290.0,73934.0,-0.22,135.77,135.99,1.91
16,470.0,463.0,448.0,523.0,JUBLFOOD.NS,903.0,468.21,109.0,SS,ATH,51034.89,51230.0,98427.0,0.38,92.86,92.48,3.11
29,396.0,383.0,376.0,395.0,WIPRO.NS,713.0,382.90,130.0,2T3Y,2T3Y,49777.00,51480.0,92690.0,3.42,86.21,82.79,3.03
14,624.0,621.0,605.0,662.0,IRCTC.NS,1164.0,629.08,80.0,SS,ATH,50326.40,49920.0,93120.0,-0.81,85.03,85.84,3.07
10,1787.0,1805.0,1766.0,1966.0,HDFCAMC.NS,3229.0,1856.05,27.0,V40,ATH,50113.35,48249.0,87183.0,-3.72,73.97,77.69,3.05


In [11]:
tmp_df.sort_values(by = 'Juice Left%', ascending=True)

,Close,20_SMA,50_SMA,200_SMA,Stock,Max,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Estimated,Current P/L%,Estimated P/L%,Juice Left%,Weightage%
6,3545.0,3422.0,3199.0,3328.0,EICHERMOT.NS,3850.0,3336.21,15.0,V40N,ATH,50043.15,53175.0,57750.0,6.26,15.40,9.14,3.05
24,766.0,765.0,726.0,757.0,TATACONSUM.NS,866.0,758.64,65.0,SS,ATH,49311.60,49790.0,56290.0,0.97,14.15,13.18,3.00
23,2455.0,2439.0,2358.0,2473.0,RELIANCE.NS,2811.0,2464.49,21.0,V40,ATH,51754.29,51555.0,59031.0,-0.39,14.06,14.45,3.15
20,2535.0,2488.0,2406.0,2545.0,PIDILITIND.NS,2902.0,2442.09,20.0,V40,ATH,48841.80,50700.0,58040.0,3.80,18.83,15.03,2.97
1,3085.0,3017.0,2897.0,3072.0,ASIANPAINT.NS,3566.0,2954.17,17.0,V40,ATH,50220.89,52445.0,60622.0,4.43,20.71,16.28,3.06
2,6793.0,6477.0,6073.0,6611.0,BAJFINANCE.NS,7901.0,6365.23,16.0,V40,ATH,101843.68,108688.0,126416.0,6.72,24.13,17.41,6.20
21,51.0,50.0,49.0,46.0,PNB.NS,60.0,50.60,600.0,NaN,NaN,30360.00,30600.0,36000.0,0.79,18.58,17.79,1.85
26,34.0,30.0,27.0,26.0,UJJIVANSFB.NS,42.0,33.60,900.0,V40N,NaN,30240.00,30600.0,37800.0,1.19,25.00,23.81,1.84
7,172.0,167.0,151.0,145.0,GICRE.NS,223.0,171.85,175.0,2T3Y,2T3Y,30073.75,30100.0,39025.0,0.09,29.76,29.67,1.83
19,41788.0,41341.0,39090.0,44208.0,PAGEIND.NS,53631.0,39840.50,2.0,V40,ATH,79681.00,83576.0,107262.0,4.89,34.61,29.72,4.85


In [12]:
tmp_df.sort_values(by = 'Weightage%', ascending=False)

,Close,20_SMA,50_SMA,200_SMA,Stock,Max,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Estimated,Current P/L%,Estimated P/L%,Juice Left%,Weightage%
12,1409.0,1255.0,1133.0,1312.0,INDIGOPNTS.NS,3112.0,1312.07,166.0,V40N,2T3Y,217803.62,233894.0,516592.0,7.39,137.18,129.79,13.27
2,6793.0,6477.0,6073.0,6611.0,BAJFINANCE.NS,7901.0,6365.23,16.0,V40,ATH,101843.68,108688.0,126416.0,6.72,24.13,17.41,6.20
4,1531.0,1506.0,1452.0,1652.0,BATAINDIA.NS,2173.0,1501.51,67.0,V40,ATH,100601.17,102577.0,145591.0,1.96,44.72,42.76,6.13
19,41788.0,41341.0,39090.0,44208.0,PAGEIND.NS,53631.0,39840.50,2.0,V40,ATH,79681.00,83576.0,107262.0,4.89,34.61,29.72,4.85
8,4375.0,4491.0,4430.0,4898.0,GILLETTE.NS,5944.0,4479.47,14.0,V40,ATH,62712.58,61250.0,83216.0,-2.33,32.69,35.02,3.82
5,3425.0,3534.0,3464.0,3900.0,DMART.NS,5324.0,3702.36,14.0,SS,ATH,51833.04,47950.0,74536.0,-7.49,43.80,51.29,3.16
23,2455.0,2439.0,2358.0,2473.0,RELIANCE.NS,2811.0,2464.49,21.0,V40,ATH,51754.29,51555.0,59031.0,-0.39,14.06,14.45,3.15
17,1445.0,1435.0,1324.0,1581.0,LUXIND.NS,4546.0,1499.87,34.0,V40N,2T3Y,50995.58,49130.0,154564.0,-3.66,203.09,206.75,3.11
16,470.0,463.0,448.0,523.0,JUBLFOOD.NS,903.0,468.21,109.0,SS,ATH,51034.89,51230.0,98427.0,0.38,92.86,92.48,3.11
11,1093.0,1091.0,1085.0,1162.0,ICICIGI.NS,1620.0,1106.17,46.0,V40,ATH,50883.82,50278.0,74520.0,-1.19,46.45,47.64,3.10


In [13]:
tmp_df.to_csv('/content/drive/My Drive/data/stocks/Stock-Portfolio-Analysis.csv', index=False)